In [1]:
import pandas as pd
import numpy as np
import joblib

## Read Data

In [13]:
data = pd.read_csv("data/linda_neck_cheek_noisy-cleaned.csv")

In [14]:
emg_data = data[['bandpass EMG envelope 0', 'bandpass EMG envelope 1']]
emg_data = emg_data.to_numpy()

button_data = data[['button 0', 'button 1']]
button_data = button_data.to_numpy()

In [15]:
def create_overlapping_sliding_windows(data, window_size, overlap):
    windows = []
    for i in range(0, len(data) - window_size + 1, overlap):
        windows.append(data[i:i+window_size, :])
    return np.array(windows)

In [16]:
windowed_emg_data = create_overlapping_sliding_windows(emg_data, 100, 40)
windowed_button_labels = create_overlapping_sliding_windows(button_data, 100, 40)

In [17]:
X = np.sqrt(np.mean(np.square(windowed_emg_data), axis=1))
y = np.max(windowed_button_labels, axis=1)

In [18]:
RMS_EMG_0 = X[:,0].reshape(-1, 1)
button_0 = y[:, 0]

RMS_EMG_1 = X[:,1].reshape(-1, 1)
button_1 = y[:, 1]

## Load Model & Make Predictions

In [19]:
emg_0_model = joblib.load("saved_models/emg_0_model.pkl")
emg_1_model = joblib.load("saved_models/emg_1_model.pkl")

In [23]:
from sklearn.model_selection import cross_validate
# from sklearn.linear_model import LogisticRegression

cv_results_0 = cross_validate(emg_1_model, RMS_EMG_0, button_0, cv=5, scoring='accuracy', return_estimator=True)
cv_results_1 = cross_validate(emg_0_model, RMS_EMG_1, button_1, cv=5, scoring='accuracy', return_estimator=True)

In [21]:
cv_results_0

{'fit_time': array([0.00562072, 0.00219727, 0.00250387, 0.00253248, 0.0023489 ]),
 'score_time': array([0.0006206 , 0.00058866, 0.00052071, 0.0006628 , 0.00058031]),
 'estimator': [LogisticRegression(),
  LogisticRegression(),
  LogisticRegression(),
  LogisticRegression(),
  LogisticRegression()],
 'test_score': array([0.89655172, 0.90697674, 0.90697674, 0.6744186 , 0.89534884])}

In [24]:
cv_results_0

{'fit_time': array([0.00339985, 0.00352859, 0.00339437, 0.00327134, 0.00370884]),
 'score_time': array([0.00098801, 0.00071263, 0.00077057, 0.00082397, 0.00082088]),
 'estimator': [LogisticRegression(),
  LogisticRegression(),
  LogisticRegression(),
  LogisticRegression(),
  LogisticRegression()],
 'test_score': array([0.89655172, 0.90697674, 0.90697674, 0.6744186 , 0.89534884])}

In [22]:
cv_results_1

{'fit_time': array([0.00230718, 0.00220728, 0.00201249, 0.00195122, 0.00211644]),
 'score_time': array([0.00076509, 0.0005095 , 0.00048733, 0.0004909 , 0.00047922]),
 'estimator': [LogisticRegression(),
  LogisticRegression(),
  LogisticRegression(),
  LogisticRegression(),
  LogisticRegression()],
 'test_score': array([0.94252874, 0.93023256, 0.89534884, 1.        , 0.91860465])}

In [25]:
cv_results_1

{'fit_time': array([0.00340605, 0.00328922, 0.00302148, 0.0027914 , 0.00345635]),
 'score_time': array([0.00067401, 0.000772  , 0.00067115, 0.00073504, 0.00089526]),
 'estimator': [LogisticRegression(),
  LogisticRegression(),
  LogisticRegression(),
  LogisticRegression(),
  LogisticRegression()],
 'test_score': array([0.94252874, 0.93023256, 0.89534884, 1.        , 0.91860465])}